In [1]:
import tensorflow as tf

2023-12-22 13:10:02.136931: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 13:10:02.181994: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 13:10:02.182018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 13:10:02.182816: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-22 13:10:02.191292: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-22 13:10:02.192544: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
data_path = "dataset"
train_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=60

)
validation_data = tf.keras.utils.audio_dataset_from_directory(
    data_path,
    batch_size=8,
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    output_sequence_length=48000,
    ragged=False,
    label_mode="categorical",
    labels="inferred",
    sampling_rate=None,
    seed=5
)


Found 1619 files belonging to 17 classes.
Using 1296 files for training.
Found 1619 files belonging to 17 classes.
Using 323 files for validation.


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=80, activation='relu', strides=16, input_shape=(48000, 1)),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(4),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(17, activation='softmax')
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2996, 32)          2592      
                                                                 
 max_pooling1d (MaxPooling1  (None, 749, 32)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 747, 32)           3104      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 186, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 184, 32)           3104      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 46, 32)            0

In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
model.fit(train_data, validation_data=validation_data, epochs=30)

Epoch 1/30


  7/162 [>.............................] - ETA: 4s - loss: 2.8199 - accuracy: 0.0714  

2023-12-22 13:10:34.633422: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 20914176 exceeds 10% of free system memory.
2023-12-22 13:10:34.633487: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 20914176 exceeds 10% of free system memory.
2023-12-22 13:10:34.635019: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 22947840 exceeds 10% of free system memory.
2023-12-22 13:10:34.635069: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 22947840 exceeds 10% of free system memory.
2023-12-22 13:10:34.640426: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23009280 exceeds 10% of free system memory.


162/162 [==============================] - 5s 25ms/step - loss: 2.5007 - accuracy: 0.1497 - val_loss: 2.2402 - val_accuracy: 0.2074
Epoch 2/30
162/162 [==============================] - 4s 23ms/step - loss: 1.8812 - accuracy: 0.3735 - val_loss: 1.4689 - val_accuracy: 0.5263
Epoch 3/30
162/162 [==============================] - 4s 24ms/step - loss: 1.2359 - accuracy: 0.5656 - val_loss: 1.0736 - val_accuracy: 0.6254
Epoch 4/30
162/162 [==============================] - 4s 22ms/step - loss: 0.8858 - accuracy: 0.7083 - val_loss: 0.9481 - val_accuracy: 0.6904
Epoch 5/30
162/162 [==============================] - 4s 21ms/step - loss: 0.7364 - accuracy: 0.7469 - val_loss: 0.7439 - val_accuracy: 0.7554
Epoch 6/30
162/162 [==============================] - 3s 21ms/step - loss: 0.6025 - accuracy: 0.8032 - val_loss: 0.5994 - val_accuracy: 0.8111
Epoch 7/30
162/162 [==============================] - 4s 24ms/step - loss: 0.5567 - accuracy: 0.8248 - val_loss: 0.5869 - val_accuracy: 0.8235
Epoch 8/30

In [9]:
model.save('audio_friends.h5')